In [1]:
# Py Torch 60 min Tutorial: NEURAL NETWORKS
# using torch.nn : nn depends on autograd to define models and differentiate
# them. An nn.module contains layers, method forward(input) that
# returns output.

# feed forward network : takes the input, feeds it through several layers 
# gives the output

# steps:
# 1) define the neural network that has some parameters
# 2) iterate over dataset of inputs
# 3) process input through the network
# 4) compute the loss
# 5) propogate the gradients into the networks parameters
# 6) update the weights of the network, 
#    using weight = weight - learning rate * gradient



# DEFINE THE NETWORK

import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channel, 5 x 5 convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # affine ops: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [2]:
# define forward func, and backward func (gradients are computed) is
# automatically defined by using autograd
# learnable params of the model are returned by net.parameters()

params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight



10
torch.Size([6, 1, 5, 5])


In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0170,  0.1856,  0.1030, -0.0840, -0.0489, -0.0677, -0.0080,  0.0301,
         -0.1125,  0.1730]], grad_fn=<AddmmBackward>)


In [4]:
# zero the gradient buffer of all the params and backprops with random gradients
net.zero_grad()
out.backward(torch.randn(1, 10))

In [5]:
# LOSS FUNCTION: takes the output, targ pair of inputs and computes a value
# that esimates how far away output is from the target

# nn.MSELoss : computes the mean squared error between inp, targ

output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)


tensor(0.7125, grad_fn=<MseLossBackward>)


In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [8]:
# BACKPROP
# all you have to do is call loss.backward(). Need to clear
# existing gradients though, else they will be accumulated to existing gradients


net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)


conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0129,  0.0017, -0.0205, -0.0107, -0.0134,  0.0098])


In [ ]:
# UPDATE THE WEIGHTS 
# simplest rule: weight = weight - learning rate * gradient

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)
    
# various other rules: sgd, nesterov-sgd, rmsprop... etc...
# torch.optim : implements these methods

import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update